<a href="https://colab.research.google.com/github/safeai-snu/Econometrics/blob/main/Part.3/8.SARIMAX_VAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 예제 8-1. SARIMAX 모델 예시

### 호주 전력 소비량 데이터 SARIMAX 모델 적합

#### 필요한 패키지 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

import warnings
warnings.filterwarnings('ignore')

#### 데이터 불러오기

In [ ]:
vic_elec = pd.read_csv("https://raw.githubusercontent.com/safeai-snu/Econometrics/refs/heads/main/Part.3/data/vic_elec.csv")

vic_elec['Time'] = pd.to_datetime(vic_elec['Time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
vic_elec['Date'] = vic_elec['Time'].dt.date

vic_elec = vic_elec[(pd.to_datetime(vic_elec['Date']).dt.year == 2013) | (pd.to_datetime(vic_elec['Date']).dt.year == 2014)]

#### 일별 데이터로의 전처리

In [ ]:
vic_elec_daily = vic_elec.groupby('Date').agg({
    'Demand': lambda x: x.sum() / 1e3,
    'Temperature': 'max',
    'Holiday': 'any'
}).reset_index()

vic_elec_daily['Day_Type'] = np.where(
    vic_elec_daily['Holiday'], 'Holiday',
    np.where(pd.to_datetime(vic_elec_daily['Date']).dt.weekday.isin([0, 1, 2, 3, 4]), 'Weekday', 'Weekend')
)

vic_elec_daily.index = vic_elec_daily['Date'].values
vic_elec_daily = vic_elec_daily[['Demand', 'Temperature', 'Day_Type']]
vic_elec_daily

#### 데이터 시각화

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(vic_elec_daily['Demand'], label='Demand')
ax1.set_ylabel('Demand (GW)')
ax1.set_title('Electricity Demand Over Time')

ax2.plot(vic_elec_daily['Temperature'], label='Temperature', color='orange')
ax2.set_ylabel('Temperature (°C)')
ax2.set_title('Maximum Daily Temperature Over Time')

ax2.set_xlabel('Date')

plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()
plt.show()

#### 외생 변수 영향 분석

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(vic_elec_daily[vic_elec_daily["Day_Type"] != 'Weekday']["Temperature"], vic_elec_daily[vic_elec_daily["Day_Type"] != 'Weekday']["Demand"], color="g", label="Playday")
plt.scatter(vic_elec_daily[vic_elec_daily["Day_Type"] == 'Weekday']["Temperature"], vic_elec_daily[vic_elec_daily["Day_Type"] == 'Weekday']["Demand"], color="b", label="Weekday")

plt.xlabel('Maximum daily temperature (°C)')
plt.ylabel('Electricity demand (GW)')
plt.title('Electricity Demand vs. Temperature by Day Type')

plt.grid(True)
plt.show()

#### Adfuller test

In [ ]:
print(adfuller(vic_elec_daily["Demand"][:-9])[1])

#### 이차항 생성

In [ ]:
X = pd.DataFrame()
X['Temperature'] = vic_elec_daily['Temperature']
X["Temperature^2"] = vic_elec_daily["Temperature"] ** 2
X["Weekday"] = (vic_elec_daily['Day_Type'] == 'Weekday').astype(int)

#### SARIMA 파라미터 최적화 함수

In [ ]:
def SARIMAX_optimizer(y, X):
    result= []
    for p in range(4):
        for q in range(4):
            for P in range(6):
                for Q in range(6-P):
                    try:
                        model = SARIMAX(y, X, order=(p, 1, q), simple_differencing=False, seasonal_order=(P, 0, Q, 7)).fit(dips=False)
                    except:
                        continue

                    aic = model.aic
                    result.append([p, q, P, Q, aic])

    result_df = pd.DataFrame(result)
    result_df.columns = ['p', 'q', 'P', 'Q', 'AIC']
    result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)

    return result_df

#### 오차 차수 최적화

In [ ]:
result = SARIMAX_optimizer(vic_elec_daily["Demand"][:-9], X[:-9])
result

#### SARIMAX 모델 적합

In [ ]:
model = SARIMAX(vic_elec_daily["Demand"][:-9], exog=X[:-9], order=(3, 0, 2), simple_differencing=False, seasonal_order=(2, 0, 2, 7))
fit = model.fit(disp=False)
print(fit.summary())

#### 잔차 확인

In [ ]:
fit.plot_diagnostics(figsize=(10,8))
plt.tight_layout()
plt.show()

#### SARIMAX 예측

In [ ]:
forecast_result = fit.get_forecast(steps=9, exog=X[-9:])
forecast_mean = forecast_result.predicted_mean
lower_ci80 = forecast_result.conf_int(alpha=0.2).iloc[:, 0]
upper_ci80 = forecast_result.conf_int(alpha=0.2).iloc[:, 1]
lower_ci95 = forecast_result.conf_int(alpha=0.05).iloc[:, 0]
upper_ci95 = forecast_result.conf_int(alpha=0.05).iloc[:, 1]

#### 예측 시각화

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(vic_elec_daily['Demand'][:-9], label='Observation', color='black')
plt.plot(forecast_mean, label='Forecast', color='b')
plt.fill_between(forecast_mean.index, lower_ci80, upper_ci80, label='80% CI', color='b', alpha=.2)
plt.fill_between(forecast_mean.index, lower_ci95, upper_ci95, label='95% CI', color='b', alpha=.1)
plt.xlabel('Date')
plt.ylabel('GW')
plt.title('Daily Electricity Demand')
plt.grid(True)
plt.legend()
plt.show()

## 예제 8-2. VAR 모델 예시

### 미국 가처분 소득 변화와 실질 소비 변화 데이터 VAR 모델 적합 후 예측

#### 필요한 패키지 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import grangercausalitytests

import warnings
warnings.filterwarnings('ignore')

#### 데이터 불러오기

In [ ]:
macro_econ_data = sm.datasets.macrodata.load_pandas().data
dpi_cons_quarter = macro_econ_data[['realdpi', 'realcons']]
dpi_cons_quarter.index = pd.date_range(start='1959/01/01', end='2009/09/30', freq='3MS')
dpi_cons_quarter

#### 데이터 시각화

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(dpi_cons_quarter['realdpi'],  color='b', label='Real DPI')
plt.plot(dpi_cons_quarter['realcons'],  color='g', label='Real Consumption')
plt.xlabel("Date")
plt.ylabel("Value (k$)")
plt.title("Real Disposable Income and Real Consumption Over Time")
plt.grid(True)
plt.legend()
plt.show()

#### 1차 차분 데이터 시각화

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(dpi_cons_quarter['realdpi'].diff()[1:], label='Real DPI')
ax1.set_ylabel('Value (k$)')
ax1.set_title('Differentiate Real Disposable Income Over Time')

ax2.plot(dpi_cons_quarter['realcons'].diff()[1:], label='Real Consumption', color='orange')
ax2.set_ylabel('Value (k$)')
ax2.set_title('Differentiate Real Consumption Over Time')

ax2.set_xlabel('Date')

plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()

#### Adfuller test

In [ ]:
print(adfuller(dpi_cons_quarter['realdpi'].diff()[1:][:180])[1])
print(adfuller(dpi_cons_quarter['realcons'].diff()[1:][:180])[1])

#### VAR 파라미터 최적화 함수

In [ ]:
def VAR_optimizer(series):
    result= []
    for p in range(10):
        try:
            model = VARMAX(series, order=(p, 0)).fit(dips=False)
        except:
            continue

        aic = model.aic
        result.append([p, aic])

    result_df = pd.DataFrame(result)
    result_df.columns = ['p', 'AIC']
    result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)

    return result_df

#### VAR 파라미터 최적화

In [ ]:
result = VAR_optimizer(dpi_cons_quarter.diff()[1:][:180])
result

#### VAR 파라미터 최적화 시각화

In [ ]:
result = result.sort_values(by='p', ascending=True).reset_index(drop=True)
plt.figure(figsize=(10, 6))
plt.plot(result['p'], result['AIC'], color='b', )
plt.xlabel("Order")
plt.ylabel("AIC")
plt.title("VAR Optimize")
plt.show()

#### 그레인저 인과관계 테스트

In [ ]:
print(grangercausalitytests(dpi_cons_quarter[['realdpi', 'realcons']].diff()[1:][:180], [3]))
print(grangercausalitytests(dpi_cons_quarter[['realcons', 'realdpi']].diff()[1:][:180], [3]))

#### VAR 모델 적합

In [ ]:
model = VARMAX(dpi_cons_quarter.diff()[1:][:180], order=(3, 0))
fit = model.fit()
print(fit.summary())

#### 가처분 소득 데이터 VAR 모델 적합 후 잔차 확인

In [ ]:
fit.plot_diagnostics(figsize=(10,8), variable=0)
plt.tight_layout()
plt.show()

#### 미국 실질 소비 데이터 VAR 모델 적합 후 잔차 확인

In [ ]:
fit.plot_diagnostics(figsize=(10,8), variable=1)
plt.tight_layout()
plt.show()

#### VAR 모델 예측

In [ ]:
forecast_result = fit.get_forecast(steps=22)

forecast_mean = forecast_result.predicted_mean
cum_forecast_result = forecast_mean.cumsum()
cum_forecast_result['realdpi'] = cum_forecast_result['realdpi'] + dpi_cons_quarter.iloc[179, 0]
cum_forecast_result['realcons'] = cum_forecast_result['realcons'] + dpi_cons_quarter.iloc[179, 1]

#### 가처분 소득 데이터 신뢰구간

In [ ]:
lower_ci80 = forecast_result.conf_int(alpha=0.2).iloc[:, 0]
upper_ci80 = forecast_result.conf_int(alpha=0.2).iloc[:, 2]
lower_ci95 = forecast_result.conf_int(alpha=0.05).iloc[:, 0]
upper_ci95 = forecast_result.conf_int(alpha=0.05).iloc[:, 2]

#### 가처분 소득 예측 시각화

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(dpi_cons_quarter['realdpi'].diff()[1:], label='Observation', color='black')
ax1.plot(forecast_mean['realdpi'], label='forcast', color='b')
ax1.fill_between(forecast_mean.index, lower_ci80, upper_ci80, label='80% CI', color='b', alpha=.2)
ax1.fill_between(forecast_mean.index, lower_ci95, upper_ci95, label='95% CI', color='b', alpha=.1)
ax1.set_ylabel('Value (k$)')
ax1.set_title('Forcasting Differentiate Real Disposable Income Over Time')

ax2.plot(dpi_cons_quarter['realdpi'], label='Observation', color='black')
ax2.plot(cum_forecast_result['realdpi'], label='forcast', color='b')
ax2.set_ylabel('Value (k$)')
ax2.set_title('Forcasting Real Disposable Income Over Time')

ax2.set_xlabel('Date')

plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
plt.legend()
plt.grid(True)
plt.tight_layout()

#### 미국 실질소비 소득 데이터 신뢰구간

In [ ]:
lower_ci80 = forecast_result.conf_int(alpha=0.2).iloc[:, 1]
upper_ci80 = forecast_result.conf_int(alpha=0.2).iloc[:, 3]
lower_ci95 = forecast_result.conf_int(alpha=0.05).iloc[:, 1]
upper_ci95 = forecast_result.conf_int(alpha=0.05).iloc[:, 3]

#### 미국 실질소비 소득 예측 시각화

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(dpi_cons_quarter['realcons'].diff()[1:], label='Observation', color='black')
ax1.plot(forecast_mean['realcons'], label='forcast', color='b')
ax1.fill_between(forecast_mean.index, lower_ci80, upper_ci80, label='80% CI', color='b', alpha=.2)
ax1.fill_between(forecast_mean.index, lower_ci95, upper_ci95, label='95% CI', color='b', alpha=.1)
ax1.set_ylabel('Value (k$)')
ax1.set_title('Forcasting Differentiate Real Consumption Over Time')

ax2.plot(dpi_cons_quarter['realcons'], label='Observation', color='black')
ax2.plot(cum_forecast_result['realcons'], label='forcast', color='b')
ax2.set_ylabel('Value (k$)')
ax2.set_title('Forcasting Real Consumption Over Time')

ax2.set_xlabel('Date')

plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
plt.legend()
plt.grid(True)
plt.tight_layout()